for the identification

In [1]:
import cv2
from datetime import datetime
import os
from ultralytics import YOLO
import pytesseract
from PIL import Image
import numpy as np

In [ ]:
5

# Load the trained model
model = YOLO("./models/NewValidator(1).pt")

# Set up video capture (0 for USB webcam or Raspberry Pi Camera)
cap = cv2.VideoCapture(0)

# To hold the data temporarily
predictions = []

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image")
        break

    # Display the current frame
    cv2.imshow('Camera Feed - Press "s" to simulate IoT signal', frame)
    
    # Check for keyboard input
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('s'):  # Simulate IoT signal with the "s" key
        print("IoT Signal Simulated with 's' key: Running YOLOv8 for object detection...")

        # Make prediction on the current frame
        results = model(frame)

        # Check if results contain data
        if len(results) > 0:
            r = results[0]  # Get the first result
            
            for box in r.boxes:
                # Extract bounding box coordinates
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())  # Convert coordinates to integers
                confidence = box.conf.cpu().item()  # Confidence score
                class_id = int(box.cls.cpu().item())  # Class ID
                class_name = r.names[class_id]  # Class name
                
                # Crop the object from the frame
                cropped_object = frame[y1:y2, x1:x2]
                
                # Save the cropped image
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                cropped_image_path = f"cropped_objects/{class_name}_{timestamp}.jpg"
                os.makedirs("cropped_objects", exist_ok=True)
                cv2.imwrite(cropped_image_path, cropped_object)
                
                print(f"Saved cropped object: {cropped_image_path}, Confidence: {confidence}")

                if confidence > 0.96:
                    text = f"{class_name} {confidence:.3f}"
                else:
                    text = "Unknown"
                
                image = frame
                cv2.rectangle(image , (x1,y1) , (x2,y2) , (0,255,0) , 2)
                cv2.putText(image , text ,(x1 , y1-10), cv2.FONT_HERSHEY_COMPLEX , 0.5 ,  (0,144,255) , 1 , cv2.LINE_AA )
                cv2.imshow('YOLOv8 Detection', image) 

    # Press 'q' to quit the program
    if key == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


Error: Could not open camera.
Error: Failed to capture image


: 

for text extraction

In [28]:
# Load the trained model
model = YOLO("./models/best (1).pt")

# Set up video capture (0 for USB webcam or Raspberry Pi Camera)
cap = cv2.VideoCapture(0)

counter = 0

# To hold the data temporarily
predictions = []

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image")
        break

    # Display the current frame
    cv2.imshow('Camera Feed - Press "s" to simulate IoT signal', frame)
    
    # Check for keyboard input
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('s'):  # Simulate IoT signal with the "s" key
        print("IoT Signal Simulated with 's' key: Running YOLOv8 for object detection...")

        # Make prediction on the current frame
        results = model(frame)

        # Check if results contain data
        if len(results) > 0:
            r = results[0]  # Get the first result
            
            for box in r.boxes:
                print("bouding box",box)
                # Extract bounding box coordinates
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())  # Convert coordinates to integers
                confidence = box.conf.cpu().item()  # Confidence score
                class_id = int(box.cls.cpu().item())  # Class ID
                class_name = r.names[class_id]  # Class name
                
                # Crop the object from the frame
                cropped_object = frame[y1:y2, x1:x2]
                
                # Save the cropped image
                counter += 1
                cropped_image_path = f"cropped_objects/{class_name}_{counter}.jpg"
                os.makedirs("cropped_objects", exist_ok=True)
                cv2.imwrite(cropped_image_path, cropped_object)
                image = cv2.imread(cropped_image_path)
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                sharpened = cv2.GaussianBlur(gray, (0, 0), 3)
                sharpened = cv2.addWeighted(gray, 1.5, sharpened, -0.5, 0)
                text = pytesseract.image_to_string(sharpened)
                print(text)
                
                print(f"Saved cropped object: {cropped_image_path}, Confidence: {confidence}")
                
                # Annotate and display the frame
                annotated_frame = results[0].plot()
                cv2.imshow('YOLOv8 Detection', annotated_frame) 

    # Press 'q' to quit the program
    if key == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


IoT Signal Simulated with 's' key: Running YOLOv8 for object detection...

0: 480x640 3 Infos, 406.0ms
Speed: 3.9ms preprocess, 406.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
bouding box ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.8019])
data: tensor([[198.2656, 342.6316, 459.8701, 402.0066,   0.8019,   0.0000]])
id: None
is_track: False
orig_shape: (480, 640)
shape: torch.Size([1, 6])
xywh: tensor([[329.0679, 372.3191, 261.6045,  59.3750]])
xywhn: tensor([[0.5142, 0.7757, 0.4088, 0.1237]])
xyxy: tensor([[198.2656, 342.6316, 459.8701, 402.0066]])
xyxyn: tensor([[0.3098, 0.7138, 0.7185, 0.8375]])
3674 9808 3213

Saved cropped object: cropped_objects/Info_1.jpg, Confidence: 0.8018737435340881
bouding box ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.7759])
data: tensor([[295.8041, 276.2598, 355.9716, 304.5311,   0.7759,   0.0000]])
id: None
is_track: False
orig_shape: (

In [18]:
from PIL import Image
import numpy as np

In [33]:
def parse_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sharpened = cv2.GaussianBlur(gray, (0, 0), 3)
    sharpened = cv2.addWeighted(gray, 1.5, sharpened, -0.5, 0)
    text1 = pytesseract.image_to_string(sharpened)
    

    # Step 2: Convert to Grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Step 3: Apply GaussianBlur to Reduce Noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Step 4: Apply Thresholding
    # You can experiment with different thresholding techniques
    # Binary Thresholding
    # _, binary_image = cv2.threshold(blurred, 150, 255, cv2.THRESH_OTSU)

    # Adaptive Thresholding
    adaptive_threshold = cv2.adaptiveThreshold(blurred, 255, 
                                            cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                            cv2.THRESH_BINARY, 11, 2)

    # Step 5: Sharpen the Image
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
    sharpened = cv2.morphologyEx(adaptive_threshold, cv2.MORPH_CLOSE, kernel)

    # Step 6: Resize the Image
    resized_image = cv2.resize(sharpened, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)

    custom_config = r'--oem 3 --psm 6'  # OCR Engine Mode 3, Page Segmentation Mode 6
    text2 = pytesseract.image_to_string(resized_image, config=custom_config)
    
    return text1,text2

In [5]:
# Load the image and mode
image_path = "./new_aadhar_card_ocr/train/images/0c0584201ff552c4bdcbe160315aa432_jpg.rf.96da227a111410ee00fc2f6ef73ba432.jpg"
model_path = "./models/NewValidator(1).pt"
image = Image.open(image_path)
model = YOLO(model_path)
results = model(image)
counter = 0
image = np.array(image)

# Ensure the directory exists
os.makedirs('./cropped_objects', exist_ok=True)

# Check if results contain data
if len(results) > 0:
    r = results[0]  # Get the first result

    for box in r.boxes:
        # Extract bounding box coordinates
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())  # Convert coordinates to integers
        confidence = box.conf.cpu().item()  # Confidence score
        class_id = int(box.cls.cpu().item())  # Class ID
        class_name = r.names[class_id]  # Class name

        # Check if the bounding box is within the image bounds
        if x1 >= 0 and y1 >= 0 and x2 <= image.shape[1] and y2 <= image.shape[0]:
            # Crop the object from the frame
            cropped_object = image[y1:y2, x1:x2]

            # Check if the cropped object is not empty
            if cropped_object.size > 0:
                # Save the cropped image with a unique filename
                counter += 1
                timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
                unique_filename = f"{class_name}_{counter}_{timestamp}.jpg"
                cropped_image_path = os.path.join('./cropped_objects', unique_filename)
                cv2.imwrite(cropped_image_path, cv2.cvtColor(cropped_object, cv2.COLOR_RGB2BGR))

                # Read the cropped image using OpenCV
                TEXT1,TEXT2 = parse_image(cropped_image_path)

                print(f"Saved cropped object: {cropped_image_path}, Confidence: {confidence}")
                print(f"Extracted Text: {TEXT1}  {TEXT2}")
            else:
                print(f"Skipping empty cropped object for class: {class_name}")
        else:
            print(f"Bounding box out of image bounds for class: {class_name}")
else:
    print("No objects detected in the image.")



0: 640x640 1 Aadhar Card, 361.0ms
Speed: 2.0ms preprocess, 361.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


NameError: name 'parse_image' is not defined

In [1]:
import easyocr
reader = easyocr.Reader(['en'])
print("EasyOCR installed successfully!")


ModuleNotFoundError: No module named 'easyocr'

In [11]:
image_path = "./10th_marksheet_entity_detection.v3i.yolov11/train/images/10th_marksheet_4_jpg.rf.55295fc9a8e0ca7f5a6f5c39afaa24df.jpg"
model_path = "./models/results.pt"


# Load the YOLOv8 model (you can replace 'yolov8n.pt' with another model like 'yolov8s.pt' or 'yolov8m.pt')
model = YOLO(model_path)

image = cv2.imread(image_path)

results = model(image)

# Process results
for result in results:
    for box in result.boxes:
        # Get bounding box coordinates
        x1, y1, x2, y2 = map(int, box.xyxy[0])  

        # Get confidence score
        confidence = box.conf[0].item()

        # Get class name
        class_id = int(box.cls[0].item())
        class_name = model.names[class_id]  # Get class name from YOLO model

        # Draw bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Put class name & confidence score on the image
        label = f"{class_name}: {confidence:.2f}"
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        print(label)

# Show the output image
cv2.imshow("YOLO Object Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 640x640 1 10 Result, 493.6ms
Speed: 4.0ms preprocess, 493.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
10 Result: 0.94


In [12]:
image = Image.open('./ronakaadha.jpg')

# Method 1: Resize to specific size
new_size = (300, 200)  # width, height
resized_image = image.resize(new_size)
print(resized_image.size)  # Output: (300, 200)
resized_image.show()

(300, 200)
